<a href="https://colab.research.google.com/github/nmq443/cognitive-science-final-project/blob/quang-branch/cwt_preprocessed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# If running on colab
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# !pip install torcheeg

In [2]:
!git clone https://github.com/nmq443/cognitive-science-final-project.git

Cloning into 'cognitive-science-final-project'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 219 (delta 18), reused 22 (delta 5), pack-reused 167
Receiving objects: 100% (219/219), 5.59 MiB | 13.19 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [3]:
import os

os.chdir('/content/cognitive-science-final-project/')

In [ ]:
from torcheeg.transforms import CWTSpectrum
import numpy as np

X = np.random.rand(288, 1, 22, 1125)
t = CWTSpectrum(contourf=True)

num_trials, num_inputs, num_eegs, num_samples = X.shape

fs = 250
freq_range = (1, 15)
freq_bins = 100
w = 5
freq = np.linspace(freq_range[0],freq_range[1],freq_bins)
widths = w * fs / (2 * freq * np.pi)

X_cwt = np.zeros((num_trials, num_eegs, widths.shape[0], num_samples))

for trial in range(num_trials):
    for in_ in range(num_inputs):
        X_cwt.append(t(eeg=X[trial, in_, :, :])['eeg'])

print(len(X_cwt))

In [4]:
import numpy as np

X = np.random.rand(288, 1, 22, 1125)
num_trials, num_inputs, num_eegs, num_samples = X.shape
print(num_trials)
print(num_inputs)
print(num_eegs)
print(num_samples)

288
1
22
1125


In [6]:
fs = 250
freq_range = (1, 15)
freq_bins = 100
w = 5
freq = np.linspace(freq_range[0],freq_range[1],freq_bins)
widths = w * fs / (2 * freq * np.pi)
print(widths.shape[0])

100


In [ ]:
import numpy as np

from torcheeg.transforms import CWTSpectrum

t = CWTSpectrum()

# X_cwt = np.zeros((num_trials, widths.shape[0], num_eegs, num_samples))

X_cwt = []
for trial in range(num_trials):
    for in_ in range(num_inputs):
        transformed = t(eeg=X[trial, in_, :, :])['eeg']
        print(transformed.shape)
        X_cwt.append(transformed)

print(X_cwt.shape)

(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128, 1125)
(22, 128

In [13]:
X_cwt = np.zeros((num_trials, widths.shape[0], num_eegs, num_samples))
print(X_cwt.shape)

(288, 100, 22, 1125)


In [17]:
import numpy as np
from scipy import signal

def morlet_wavelet_transform(X,fs=250,freq_range=(1,15),freq_bins=100,w=5):
    '''
    Discrete continous wavelet transform of eeg data convolved with complex morlet wavelet
    INPUTS:
    X - EEG data (num_trials, 1, num_eeg_electrodes, time_bins)
    fs - sampling rate in Hz
    freq_range - tuple containing min and max freq range to perform analysis within
    freq_bins - number of points between freq range being analyzed
    w - Omega0 for complex morlet wavelet
    OUTPUTS:
    X_cwt - Wavlet transformed eeg data (num_trials, num_eeg_electrodes,freq_bins,time_bins)
    '''

    N_trials, _, N_eegs, time_bins = X.shape
    # values for cwt
    freq = np.linspace(freq_range[0],freq_range[1],freq_bins)
    widths = w * fs / (2 * freq * np.pi)
    X_cwt = np.zeros((N_trials,widths.shape[0],N_eegs,time_bins))

    print('Performing discrete CWT convolutions...')
    for trial in range(N_trials):
        for eeg in range(N_eegs):
            X_cwt[trial,:,eeg,:] = np.abs(signal.cwt(np.squeeze(X[trial,:,eeg,]),signal.morlet2,widths,w=w))

    return X_cwt

In [18]:
import time
import shutil
import numpy as np
import tensorflow as tf
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from preprocess import get_data
from utils import getModel
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from evaluation import draw_learning_curves
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score

def train_model_with_cwt(dataset_conf, train_conf, results_path):

    # remove the 'result' folder before training
    if os.path.exists(results_path):
        # Remove the folder and its contents
        shutil.rmtree(results_path)
        os.makedirs(results_path)

    # Get the current 'IN' time to calculate the overall training time
    in_exp = time.time()
    # Create a file to store the path of the best model among several runs
    best_models = open(results_path + "/best models.txt", "w")
    # Create a file to store performance during training
    log_write = open(results_path + "/log.txt", "w")

    # Get dataset paramters
    dataset = dataset_conf.get('name')
    n_sub = dataset_conf.get('n_sub')
    data_path = dataset_conf.get('data_path')
    isStandard = dataset_conf.get('isStandard')
    LOSO = dataset_conf.get('LOSO')
    # Get training hyperparamters
    batch_size = train_conf.get('batch_size')
    epochs = train_conf.get('epochs')
    patience = train_conf.get('patience')
    lr = train_conf.get('lr')
    LearnCurves = train_conf.get('LearnCurves') # Plot Learning Curves?
    n_train = train_conf.get('n_train')
    model_name = train_conf.get('model')
    from_logits = train_conf.get('from_logits')

    # Initialize variables
    acc = np.zeros((n_sub, n_train))
    kappa = np.zeros((n_sub, n_train))

    # Iteration over subjects
    # for sub in range(n_sub-1, n_sub): # (num_sub): for all subjects, (i-1,i): for the ith subject.
    for sub in range(n_sub): # (num_sub): for all subjects, (i-1,i): for the ith subject.

        print('\nTraining on subject ', sub+1)
        log_write.write( '\nTraining on subject '+ str(sub+1) +'\n')
        # Initiating variables to save the best subject accuracy among multiple runs.
        BestSubjAcc = 0
        bestTrainingHistory = []

        # Get training and test data
        X_train, _, y_train_onehot, _, _, _ = get_data(
            data_path, sub, dataset, LOSO = LOSO, isStandard = isStandard)
        X_train = morlet_wavelet_transform(X_train)

        # Divide the training data into training and validation
        X_train, X_val, y_train_onehot, y_val_onehot = train_test_split(X_train, y_train_onehot, test_size=0.2, random_state=42)

        # Iteration over multiple runs
        for train in range(n_train): # How many repetitions of training for subject i.
            # Set the random seed for TensorFlow and NumPy random number generator.
            # The purpose of setting a seed is to ensure reproducibility in random operations.
            tf.random.set_seed(train+1)
            np.random.seed(train+1)

            # Get the current 'IN' time to calculate the 'run' training time
            in_run = time.time()

            # Create folders and files to save trained models for all runs
            filepath = results_path + '/saved models with cwt/run-{}'.format(train+1)
            if not os.path.exists(filepath):
                os.makedirs(filepath)
            filepath = filepath + '/subject-{}.h5'.format(sub+1)

            # Create the model
            model = getModel(model_name, dataset_conf, from_logits)
            # Compile and train the model
            model.compile(loss=CategoricalCrossentropy(from_logits=from_logits), optimizer=Adam(learning_rate=lr), metrics=['accuracy'])

            # model.summary()
            # plot_model(model, to_file='plot_model.png', show_shapes=True, show_layer_names=True)

            callbacks = [
                ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                                save_best_only=True, save_weights_only=True, mode='min'),
                ReduceLROnPlateau(monitor="val_loss", factor=0.90, patience=20, verbose=0, min_lr=0.0001),
                # EarlyStopping(monitor='val_loss', verbose=1, mode='min', patience=patience)
            ]
            history = model.fit(X_train, y_train_onehot, validation_data=(X_val, y_val_onehot),
                                epochs=epochs, batch_size=batch_size, callbacks=callbacks, verbose=0)

            # Evaluate the performance of the trained model based on the validation data
            # Here we load the Trained weights from the file saved in the hard
            # disk, which should be the same as the weights of the current model.
            model.load_weights(filepath)
            y_pred = model.predict(X_val)

            if from_logits:
                y_pred = tf.nn.softmax(y_pred).numpy().argmax(axis=-1)
            else:
                y_pred = y_pred.argmax(axis=-1)

            labels = y_val_onehot.argmax(axis=-1)
            acc[sub, train]  = accuracy_score(labels, y_pred)
            kappa[sub, train] = cohen_kappa_score(labels, y_pred)

            # Get the current 'OUT' time to calculate the 'run' training time
            out_run = time.time()
            # Print & write performance measures for each run
            info = 'Subject: {}   seed {}   time: {:.1f} m   '.format(sub+1, train+1, ((out_run-in_run)/60))
            info = info + 'valid_acc: {:.4f}   valid_loss: {:.3f}'.format(acc[sub, train], min(history.history['val_loss']))
            print(info)
            log_write.write(info +'\n')
            # If current training run is better than previous runs, save the history.
            if(BestSubjAcc < acc[sub, train]):
                 BestSubjAcc = acc[sub, train]
                 bestTrainingHistory = history

        # Store the path of the best model among several runs
        best_run = np.argmax(acc[sub,:])
        filepath = '/saved models/run-{}/subject-{}.h5'.format(best_run+1, sub+1)+'\n'
        best_models.write(filepath)

        # Plot Learning curves
        if (LearnCurves == True):
            print('Plot Learning Curves ....... ')
            draw_learning_curves(bestTrainingHistory, sub+1)

    # Get the current 'OUT' time to calculate the overall training time
    out_exp = time.time()

    # Print & write the validation performance using all seeds
    head1 = head2 = '         '
    for sub in range(n_sub):
        head1 = head1 + 'sub_{}   '.format(sub+1)
        head2 = head2 + '-----   '
    head1 = head1 + '  average'
    head2 = head2 + '  -------'
    info = '\n---------------------------------\nValidation performance (acc %):'
    info = info + '\n---------------------------------\n' + head1 +'\n'+ head2
    for run in range(n_train):
        info = info + '\nSeed {}:  '.format(run+1)
        for sub in range(n_sub):
            info = info + '{:.2f}   '.format(acc[sub, run]*100)
        info = info + '  {:.2f}   '.format(np.average(acc[:, run])*100)
    info = info + '\n---------------------------------\nAverage acc - all seeds: '
    info = info + '{:.2f} %\n\nTrain Time  - all seeds: {:.1f}'.format(np.average(acc)*100, (out_exp-in_exp)/(60))
    info = info + ' min\n---------------------------------\n'
    print(info)
    log_write.write(info+'\n')

    # Close open files
    best_models.close()
    log_write.close()

2024-05-21 18:06:09.739687: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-21 18:06:10.068503: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 18:06:10.803562: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [19]:
# If run on local machine
root_data_path = '/home/quang'
# If run on colab
# root_data_path = '/content/drive/MyDrive'

In [20]:
def run():
    # Define dataset parameters
    dataset = 'BCI2a'
    in_samples = 1125
    n_channels = 22
    n_sub = 9
    n_classes = 4
    classes_labels = ['Left hand', 'Right hand','Foot','Tongue']
    data_path = os.path.expanduser(root_data_path) + '/BCI Competition IV/BCI Competition IV-2a/BCI Competition IV 2a mat/'

    # Create a folder to store the results of the experiment
    results_path = os.getcwd() + "/results"
    if not  os.path.exists(results_path):
      os.makedirs(results_path)   # Create a new directory if it does not exist

    # Set dataset paramters
    dataset_conf = { 'name': dataset, 'n_classes': n_classes, 'cl_labels': classes_labels,
                    'n_sub': n_sub, 'n_channels': n_channels, 'in_samples': in_samples,
                    'data_path': data_path, 'isStandard': True, 'LOSO': False}
    # Set training hyperparamters
    train_conf = { 'batch_size': 64, 'epochs': 500, 'patience': 100, 'lr': 0.001,'n_train': 1,
                  'LearnCurves': True, 'from_logits': False, 'model':'ATCNet_CWT'}

    # Train the model
    train_model_with_cwt(dataset_conf, train_conf, results_path)

    # Evaluate the model based on the weights saved in the '/results' folder
    model = getModel(train_conf.get('model'), dataset_conf)
    test(model, dataset_conf, results_path)


In [ ]:
from preprocess import get_data
from train import train_model
from evaluation import test
from utils import getModel
import time
import os

run()


Training on subject  1
